# December 08, 2025

https://adventofcode.com/2025/day/8

In [1]:
from math import sqrt, prod
import pandas as pd
import numpy as np

In [2]:
test_text = f'''162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689'''

test = test_text.split("\n")


In [3]:
fn = "../data/2025/08.txt"
with open(fn, "r") as file:
    puzz_text = file.readlines()
puzz = [ line.strip() for line in puzz_text ]


# Part 1

In [4]:
class JunctionBox:
    def __init__(self, x, y=None, z=None):
        if type(x) == str:
            self.xyz = [int(val) for val in x.split(",")]
        else:
            self.xyz = [x,y,z]

    def dist(self, other):
        return sqrt( sum( [ (xi - yi)**2 for xi,yi in zip(self.xyz, other.xyz) ] ) )

In [5]:
class Playground:
    def __init__(self, text):
        self.boxes = list()
        for line in text:
            self.boxes.append( JunctionBox(line) )

        self.circuits = [int(i) for i in range(len(self.boxes))]
        
        self.distmap = list()
        for i in range(len(self.boxes)):
            for j in range(i+1, len(self.boxes)):
                self.distmap.append( (int(i), int(j), self.boxes[i].dist( self.boxes[j] )) )
        self.distmap = pd.DataFrame( self.distmap, columns=["from", "to", "dist"] )
        self.distmap.sort_values( by="dist", inplace=True)

    def link( self, i, verbose=False ):
        '''link the ith closest pair of junction boxes'''
        x,y = self.distmap.iloc[i][["to", "from"]]
        x, y = int(x), int(y)

        min_circ = min(self.circuits[x], self.circuits[y])
        max_circ = max(self.circuits[x], self.circuits[y])
        min_circ = self.circuits[x]
        max_circ = self.circuits[y]
        for i, c in enumerate(self.circuits):
            if c == max_circ:
                self.circuits[i] = min_circ
        if verbose:
            print( f"Linking {x} to {y}" )
            print( f"Circuits: {len(np.unique(self.circuits))}" )

    def circuit_sizes( self ):
        circ_count = pd.DataFrame( [ [i,c] for i,c in enumerate(self.circuits) ], columns=["box", "circuit"] )
        circ_count = circ_count.groupby("circuit").count()
        circ_count.sort_values( "box", ascending=False, inplace=True )
        return circ_count
        


In [6]:
def part1( puzz, links, top, verbose=False ):
    pg = Playground(puzz)
    for i in range(links):
        pg.link(i, verbose=verbose)

    topn = pg.circuit_sizes()
    return prod( topn["box"][:top] )


In [7]:
part1(test, 10, 3, verbose=True)


Linking 19 to 0
Circuits: 19
Linking 7 to 0
Circuits: 18
Linking 13 to 2
Circuits: 17
Linking 19 to 7
Circuits: 17
Linking 18 to 17
Circuits: 16
Linking 12 to 9
Circuits: 15
Linking 16 to 11
Circuits: 14
Linking 8 to 2
Circuits: 13
Linking 19 to 14
Circuits: 12
Linking 18 to 2
Circuits: 11


40

In [9]:
# too high: 186405
part1(puzz, 1000, 3, verbose=False)

175440

# Part 2

In [19]:
def part2( puzz, verbose=False ):
    i = 0
    Puzz = Playground( puzz )
    while len( np.unique(Puzz.circuits) ) > 1:
        Puzz.link(i, verbose=verbose)
        i += 1
        
    x, y = Puzz.distmap.iloc[i-1][["from", "to"]]
    ans = Puzz.boxes[int(x)].xyz[0] * Puzz.boxes[(int(y))].xyz[0]
    return ans



In [22]:
part2( test, verbose=False )

25272

In [23]:

part2( puzz )

3200955921